## Unsloth 활용 LLM 파인 튜닝 

In [1]:
from unsloth import FastLanguageModel

import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

#Unsloth에서 제공하는 Llama기반 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 4. Max memory: 23.553 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/home/moon/miniconda3/envs/fine/lib/python3.11/site-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #LoRA 랭크 값(LoRA가 학습할 수 있는 정보량)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # LoRA 학습에 ㅎ사용되는 스케일링 계수(r과 같거나 2배 정도로 설정), 학습 강도를 조절(값이 크면 더 많이 학습)
    lora_dropout = 0, # 드롭아웃 비율(학습 중 일부 뉴련을 무작위로 끄는 확률)
    bias = "none",    # bias 사용 여부
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # 그래디언트 체크포인팅 활성화 여부(true, False, unsloth)
    random_state = 3407, # 랜덤시드값(재현 가능한 결과를 위한 설정)
    use_rslora = False,  # Rank-Stabilized LoRA 사용 여부 
    loftq_config = None, # And LoftQ( LoRA 행렬을 weight의 상위 특이벡터로 초기화 → 정확도 향상 가능)
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)